# Importing Data

In [1]:
from msg_parsing import OrderBook
import pandas as pd
x = OrderBook('3738 HK Equity')
x.run()

100%|██████████| 1/1 [00:00<00:00,  9.20it/s]


In [3]:
x.all_msg_df

,MKTDEPTH_EVENT_TYPE,MKTDEPTH_EVENT_SUBTYPE,ID_BB_SEC_NUM_SRC,FEED_SOURCE,EID,MD_TABLE_CMD_RT,MD_BOOK_TYPE,MBL_WINDOW_SIZE,MD_BOOK_SUBTYPE,MD_PRICE_DISPLAY_FORMAT_TYPE_RT,...,MBL_ASK_RT,MBL_ASK_COND_CODE_RT,MBL_ASK_NUM_ORDERS_RT,MBL_ASK_SIZE_RT,MBO_ASK_POSITION_RT,MBO_ASK_RT,MBO_ASK_BROKER_RT,MBO_ASK_COND_CODE_RT,MBO_ASK_SIZE_RT,MBO_ORDER_ID_RT
0,MARKET_BY_LEVEL,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,MBL-RBP,10.0,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,53714,ADD,MBO-AMD,NaN,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,TOP-RBP,NaN,TOPBROKER,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARKET_BY_LEVEL,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
x.trade_prints

,time,type,value,size,conditionCodes
0,2023-06-23 09:10:00.030,TRADE,2.6,4000,IE
1,2023-06-23 09:20:28.772,TRADE,2.6,4000,U


In [11]:
pd.to_pickle(x.all_msg_df, 'res/trade_recordings/3738_HK_all_msg_df.pickle')
pd.to_pickle(x.trade_prints, 'res/trade_recordings/3738_HK_trade_prints.pickle')

# Matching Auction Orders

In [12]:
import pandas as pd
all_msg_df = pd.read_pickle('res/trade_recordings/3738_HK_all_msg_df.pickle')
trade_prints = pd.read_pickle('res/trade_recordings/3738_HK_trade_prints.pickle')

In [13]:
all_msg_df

,MKTDEPTH_EVENT_TYPE,MKTDEPTH_EVENT_SUBTYPE,ID_BB_SEC_NUM_SRC,FEED_SOURCE,EID,MD_TABLE_CMD_RT,MD_BOOK_TYPE,MBL_WINDOW_SIZE,MD_BOOK_SUBTYPE,MD_PRICE_DISPLAY_FORMAT_TYPE_RT,...,MBL_ASK_RT,MBL_ASK_COND_CODE_RT,MBL_ASK_NUM_ORDERS_RT,MBL_ASK_SIZE_RT,MBO_ASK_POSITION_RT,MBO_ASK_RT,MBO_ASK_BROKER_RT,MBO_ASK_COND_CODE_RT,MBO_ASK_SIZE_RT,MBO_ORDER_ID_RT
0,MARKET_BY_LEVEL,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,MBL-RBP,10.0,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,53714,ADD,MBO-AMD,NaN,CLOB,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MARKET_BY_ORDER,TABLE_INITPAINT,7.302263e+11,HK,14112,REPLACE,TOP-RBP,NaN,TOPBROKER,DECIMAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MARKET_BY_LEVEL,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,MARKET_BY_ORDER,BID,NaN,NaN,14112,REPLACE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
